In [1]:
import (
	"fmt"
	"image/color"
    "image"
	"gocv.io/x/gocv"
)

## convert MxNet to OpenVINO
```
python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_mxnet.py --input_model ./mnist-lenet-0010.params --input_shape [1,1,28,28] --input_symbol ./mnist-lenet-symbol.json
```

In [2]:
const deviceID = 0
const proto = "./mnist-lenet-0010.xml"
const model = "./mnist-lenet-0010.bin"
backend := gocv.NetBackendOpenVINO
target := gocv.NetTargetCPU

In [3]:
n := gocv.ReadNet(model, proto)
net := &n
if net.Empty() {
	fmt.Printf("Error reading network model from : %v %v\n", model, proto)
}

In [4]:
net.SetPreferableBackend(gocv.NetBackendType(backend))
net.SetPreferableTarget(gocv.NetTargetType(target))

fmt.Printf("Start reading device: %v\n", deviceID)

Start reading device: 0
24
<nil>


In [5]:
filename := "../data/tmp/img0.png"
img := gocv.IMRead(filename, gocv.IMReadColor)
if img.Empty() {
    panic("error to load image")
}
gray := gocv.NewMat()
gocv.CvtColor(img, &gray, gocv.ColorBGRToGray)

In [ ]:
blob := gocv.BlobFromImage(gray, 1.0/255.0, image.Pt(28, 28), gocv.NewScalar(0, 0, 0, 0), false, false)
net.SetInput(blob, "data")

// run a forward pass thru the network
prob := net.Forward("softmax")

s := prob.Size()
_,_,_, out := gocv.MinMaxLoc(prob)
fmt.Println(out)

outImgData, err := gocv.IMEncode(gocv.PNGFileExt, img)
if err!=nil {
    panic("error encode image")
}
img.Close()
_ctx.Display.Text("out", nil)
_ctx.Display.PNG(outImgData, nil)